## Parser / Python's html.parser

In [16]:
from bs4 import BeautifulSoup
html = ''' 
<ol>
    <li>NEVER - 국민의 아들</li>
    <li>SIGNAL - TWICE</li>
    <li>LONELY - 씨스타</li>
    <li>I LUV IT - PSY</li>
    <li>New Face - PSY</li>
</ol>
'''
soup = BeautifulSoup(html, 'html.parser')
for tag in soup.select('li'):
    print(tag.text)

NEVER - 국민의 아들
SIGNAL - TWICE
LONELY - 씨스타
I LUV IT - PSY
New Face - PSY


## Parser / lxml's HTML parser

In [17]:
from bs4 import BeautifulSoup
html = ''' 
<ol>
    <li>NEVER - 국민의 아들</li>
    <li>SIGNAL - TWICE</li>
    <li>LONELY - 씨스타</li>
    <li>I LUV IT - PSY</li>
    <li>New Face - PSY</li>
</ol>
'''
soup = BeautifulSoup(html, 'lxml')
for tag in soup.select('li'):
    print(tag.text)

NEVER - 국민의 아들
SIGNAL - TWICE
LONELY - 씨스타
I LUV IT - PSY
New Face - PSY


---

만들어야 하는 것 
1. ~~국가별 GDP 확인 가능한 테이블 (GDP가 높은 국가들 순서로/ 단위는 1B USD로 소수점 2자리/ 갱신해도 재사용 가능하게)~~
2. ~~로그 기록시 datetime Year-Monthname-Day-Hour-Minute-Second (각 단계의 시작과 끝을 로그에 기록(기족 파일에 append))~~
3. ~~추출 (Extract)한 정보는 'Countries_by_GDP.json'라는 이름의 JSON 화일 포맷으로 저장~~


지켜야 하는 것
1. 주석 사용
2. 함수 만들어 재사용성

화면 출력
1. ~~GDP가 100B USD이상이 되는 국가만을 구해서 화면에 출력~~
2. ~~각 Region별로 top5 국가의 GDP 평균을 구해서 화면에 출력~~


---

### 준호님 코드

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import json
from tabulate import tabulate
from datetime import datetime

# 로그 기록 함수
def log_message(message):
    with open('etl_project_log.txt', 'a') as f:
        current_time = datetime.now().strftime('%Y-%B-%d-%H-%M-%S')
        f.write(f"{current_time}, {message}\n")

# 실행 구분선 추가 함수
def log_separator():
    with open('etl_project_log.txt', 'a') as f:
        f.write("\n" + "="*50 + "\n")
        current_time = datetime.now().strftime('%Y-%B-%d-%H-%M-%S')
        f.write(f"🚀 New Execution at {current_time}\n")
        f.write("="*50 + "\n\n")

# URL 설정
url = 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)'

# 웹 페이지 요청 및 파싱
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# gdp 테이블 추출
def extract_gdp_table(soup):
    log_message("extract_gdp_table 시작")
    # 테이블 추출 (지정된 class 사용)  
    table = soup.find('table', {'class': 'wikitable'})

    # 데이터 저장용 리스트 초기화
    data = []

    # 테이블에서 모든 <tr> 태그 추출
    rows = table.find_all('tr')

    # 각 <tr>에서 <td>를 추출해 데이터 리스트에 추가
    for row in rows:
        cols = row.find_all('td')
        if len(cols) >= 3:  # 적어도 3개의 <td>가 있는 경우만 처리
            country = cols[0].get_text(strip=True)
            forecast = cols[1].get_text(strip=True)
            year = cols[2].get_text(strip=True)

            # [n 1]과 같은 태그 제거 (정규표현식 사용)
            year = re.sub(r'\[.*?\]', '', year)

            data.append([country, forecast, year])

    # 데이터프레임 생성
    df = pd.DataFrame(data, columns=['Country', 'GDP_USD_billion', 'Year'])
    log_message("extract_gdp_table 완료")

    return df

def transform_gdp_data(df):
    log_message("transform_gdp_data 시작")
    # 0번 인덱스 (World) 제거
    df = df[df['Country'] != 'World']

    # Forecast가 숫자가 아닌 경우(예: '—') 제거 (GDP 100B 이상 국가 중 쿠바가 제외되는 이슈가 있음)
    df = df[df['GDP_USD_billion'] != '—']

    # 쉼표 제거 후 숫자로 변환
    df['GDP_USD_billion'] = df['GDP_USD_billion'].str.replace(',', '').astype(int)

    # Forecast 값을 100으로 나누어 1B USD로 변환 (소수점 둘째 자리까지)
    df['GDP_USD_billion'] = (df['GDP_USD_billion'] / 1000).round(2)

    # country_region_table.json 파일로부터 데이터 로드
    with open('country_region_table.json', 'r') as f:
        region_mapping = json.load(f)

    # Region 매핑 추가
    df['Region'] = df['Country'].map(region_mapping)

    log_message("transform_gdp_data 완료")
    return df

def load_gdp_data(df):
    log_message("load_gdp_data 시작")
    # Countries_by_GDP.json 파일로 저장
    df.to_json('Countries_by_GDP.json', orient='records', indent=4)
    log_message("load_gdp_data 완료")

def print_gdp_over_100b(df):
    log_message("print_gdp_over_100b 시작")
    # GDP 100B USD 이상 국가만 필터링
    gdp_over_100b = df[df['GDP_USD_billion'] >= 100]

    # 인덱스 리셋 (1부터 시작)
    gdp_over_100b.index = range(1, len(gdp_over_100b) + 1)

    # GDP 100B 이상 국가 출력
    print("🌍 GDP 100B USD 이상 국가 목록:")
    print(tabulate(gdp_over_100b, headers='keys', tablefmt='grid'))
    log_message("print_gdp_over_100b 완료")

def print_region_avg_gdp(df):
    log_message("print_region_avg_gdp 시작")
    # Region별로 그룹화 및 상위 5개 국가 추출
    top_5_per_region = (
        df.groupby('Region')
        .apply(lambda x: x.nlargest(5, 'GDP_USD_billion'))  # 상위 5개 추출
        .reset_index(drop=True)  # 인덱스 리셋
    )
    # Region별 상위 5개 국가의 GDP 평균 계산
    region_avg_gdp = (
        top_5_per_region.groupby('Region')['GDP_USD_billion']
        .mean()
        .round(2)
        .reset_index()
        .rename(columns={'GDP_USD_billion': 'Top 5 Avg GDP'})
    )
    # 내림차순 정렬
    region_avg_gdp = region_avg_gdp.sort_values('Top 5 Avg GDP', ascending=False)

    # 인덱스 리셋 (1부터 시작)
    region_avg_gdp.index = range(1, len(region_avg_gdp) + 1)

    # Region별 상위 5개 국가 평균 GDP 출력
    print("\n📊 Region별 상위 5개 국가의 GDP 평균:")
    print(tabulate(region_avg_gdp, headers='keys', tablefmt='grid'))
    log_message("print_region_avg_gdp 완료")


def main():
    log_separator()  # 실행 구분선 추가

    # gdp 테이블 추출
    raw_df = extract_gdp_table(soup)

    # 데이터 변환
    transformed_df = transform_gdp_data(raw_df)

    # 데이터 json 형태로 저장
    load_gdp_data(transformed_df)

    # 100B 이상 국가 출력
    print_gdp_over_100b(transformed_df)

    # Region별 상위 5개 국가의 GDP 평균 출력
    print_region_avg_gdp(transformed_df)


if __name__ == '__main__':
    main()

---

### 민재님 코드

In [ ]:
import datetime
import pandas as pd
import json
from bs4 import BeautifulSoup
import requests


TARGET_URL = 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'
LOG_FILE_PATH = 'etl_project_log.txt'
OUTPUT_FILE_PATH = 'Countries_by_GDP.json'
COUNTRY_REGION_TABLE_PATH = 'country_region_table.json'

class Logger:
    '''
    Logger class
    '''
    def __init__(self, log_file_path: str):
        self.log_file_path = log_file_path

    def info(self, message: str):
        self.__log('INFO', message)

    def error(self, message: str):
        self.__log('ERROR', message)

    def __log(self, type: str, message: str):
        with open(self.log_file_path, 'a') as log_file:
            timestamp = datetime.datetime.now()
            log_data = f'{timestamp.strftime("%Y-%b-%d-%H-%M-%S")}, {type}: {message}'
            log_file.write(log_data+'\n')
            print(log_data)

logger = Logger(LOG_FILE_PATH)

def get_html(url: str) -> str:
    '''
    Fetch HTML from the given URL
    '''
    try: 
        response = requests.get(url)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        return None
    
def parse_html_to_table(html:str, target: str) -> list:
    '''
    Select HTML table by class name and parse to list
    '''
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table', class_=target)
    data = []
    try:
        if not table:
            return data
        rows = table.find_all('tr')
        for row in rows[3:]:
            columns = row.find_all('td')
            country = columns[0].text.strip()
            gdp = columns[1].text.strip()
            if (not gdp or gdp == '—'):
                continue
            data.append({'Country': country, 'GDP': gdp})
        return data
    except Exception as e:
        logger.error(f'ERROR: Error parsing HTML: {e}')
        return []

def parse_json(file_path):
    '''
    Read JSON file
    '''
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

def parse_to_df(target):
    '''
    Convert list to dataframe
    '''
    df = pd.DataFrame(target)
    return df

def transfrom_df(df):
    '''
    Transformation function
    '''
    # Million -> Billion
    df['GDP'] = df['GDP'].apply(lambda x: x.replace(',', ''))
    df['GDP'] = df['GDP'].apply(lambda x: round(float(x) / 1000, 2))
    
    # Sort by GDP
    df = df.sort_values(by='GDP', ascending=False)
    
    # Add region
    country_region_table = parse_json(COUNTRY_REGION_TABLE_PATH)
    df['Region'] = df['Country'].map(country_region_table)
    
    return df

def save_to_json(df, file_path):
    '''
    Save dataframe to JSON file
    '''
    df.to_json(file_path, orient='records', indent=2)
    
def main():
    logger.info('-'*50)
    logger.info('Starting the ETL process')
    logger.info(f'Fetching HTML from {TARGET_URL}...')
    html = get_html(TARGET_URL)
    if not html:
        logger.error(f'Failed to fetch HTML from {TARGET_URL}')
        return
    logger.info('Parsing HTML to table...')
    data = parse_html_to_table(html, 'wikitable')
    if not data:
        logger.error('Table Parsing Failed')
        return
    logger.info('Transforming data to dataframe...')
    df = parse_to_df(data)
    df = transfrom_df(df)
    
    logger.info(f'Saving data to {OUTPUT_FILE_PATH}...')
    save_to_json(df, OUTPUT_FILE_PATH)
    
    logger.info('ETL process completed successfully')
    
    # print output
    print(f'Countries with GDP > 100B: \n{df[df["GDP"] > 100]}')
    df_groupby_top5 = df.groupby('Region').head(5)
    avg_gdp = df_groupby_top5.groupby('Region')['GDP'].mean()
    print('Top 5 Average GDP by Region:')
    print(avg_gdp)
    
    

if __name__ == '__main__':
    main()

---

## 내 코드

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import json

# 로그 기록 함수
def log_message(message):
    timestamp = datetime.datetime.now().strftime('%Y-%b-%d-%H-%M-%S')
    with open('etl_project_log.txt', 'a') as log_file:
        log_file.write(f"{timestamp}, {message}\n")

# Extract
def extract_gdp_data(url):
    log_message("Starting Data Extraction")
    # HTTP 요청 후 파싱해서 데이터 추출
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'class': 'wikitable'})
    rows = table.find_all('tr')

    data = []
    for row in rows[1:]:  # 첫 번째 행은 헤더이므로 제외함
        cols = row.find_all('td')
        cleaned_cols = []
        for col in cols:
            # 'sup' 태그 제거 [n 1] 이런식으로 자꾸 떠서
            for sup in col.find_all('sup'):
                sup.decompose()
            cleaned_cols.append(col.text.strip())  # 텍스트 정리
        if cleaned_cols:  # 빈 행 제외
            try:
                country = cleaned_cols[0]  # 국가명
                gdp_raw = cleaned_cols[1]  # GDP 값 (Nominal GDP)
                gdp_year = cleaned_cols[2]
                gdp_cleaned = ''.join(filter(str.isdigit, gdp_raw.split('.')[0]))
                if gdp_cleaned:  # GDP 값이 유효한 경우만 추가
                    gdp = int(gdp_cleaned) / 1e3  # 단위를 1B USD로 변환
                    data.append({'Country': country, 'GDP (B USD)': round(gdp, 2), 'Year': gdp_year})
            except IndexError:
                # 예상치 못한 데이터 구조를 무시
                continue
    log_message("Data Extraction Completed")
    
    return data

# Transform
def transform_gdp_data(data):
    log_message("Starting Data Transformation")
    # 데이터프레임 생성
    df = pd.DataFrame(data)

    # GDP 순으로 정렬
    df = df.sort_values(by='GDP (B USD)', ascending=False)
    
    # 국가별 Region 정보 매핑
    with open('country_region_table.json', 'r', encoding='utf-8') as region_file:
        region_data = json.load(region_file)
    df['Region'] = df['Country'].map(region_data)
    
    log_message("Data Transformation Completed")
    return df

# Load
def load_gdp_data(df, output_csv_file='gdp_by_country.csv', output_json_file='Countries_by_GDP.json'):
    log_message("Starting Data Loading")
    try:
        # CSV 파일로 저장
        df.to_csv(output_csv_file, index=False)
        log_message(f"CSV file saved as {output_csv_file}")
        
        # JSON 파일로 저장
        data_as_dict = df.to_dict(orient='records')  # DataFrame을 딕셔너리 리스트로 변환
        with open(output_json_file, 'w', encoding='utf-8') as json_file:
            json.dump(data_as_dict, json_file, ensure_ascii=False, indent=4)
        log_message(f"JSON file saved as {output_json_file}")
        
        log_message("Data Loading Completed Successfully")
        
    except Exception as e:
        log_message(f"Data Loading Failed: {str(e)}")
        raise  # 예외를 다시 던져 ETL 프로세스에서 처리 가능


# GDP가 100B USD 이상인 국가만 필터링
def filtered_100USD(df):
    filtered_100 = df[df['GDP (B USD)'] >= 100]
    print("Countries with a GDP of over 100B USD")
    print(filtered_100)
    
    
 # Region별 상위 5개 국가의 GDP 평균 계산
def region_top5_calculate(df):
    region_top5_avg = (
        df.groupby('Region')
        .apply(lambda x: x.nlargest(5, 'GDP (B USD)')['GDP (B USD)'].mean())
        .reset_index(name='Top 5 Avg GDP (B USD)')
    )
    print("Average GDP of top 5 countries by region")
    print(region_top5_avg)
    

# 메인 ETL 함수
def etl_process():
    url = "https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)"
    try:
        log_message("ETL Process Started")
        # Extract
        data = extract_gdp_data(url)
        # Transform
        transformed_data = transform_gdp_data(data)
        # Load
        load_gdp_data(transformed_data)  
        log_message("ETL Process End Successfully")
        
        # GDP가 100B USD 이상인 국가만 필터링
        filtered_100USD(transformed_data)
        # Region별 상위 5개 국가의 GDP 평균 계산
        region_top5_calculate(transformed_data)
        
    except Exception as e:
        log_message(f"ETL Process Failed: {str(e)}")

if __name__ == "__main__":
    etl_process()

Countries with a GDP of over 100B USD
          Country  GDP (B USD)  Year         Region
0           World    115494.31  2025            NaN
1   United States     30337.16  2025  North America
2           China     19534.89  2025           Asia
3         Germany      4921.56  2025         Europe
4           Japan      4389.33  2025           Asia
..            ...          ...   ...            ...
68     Uzbekistan       112.65  2024           Asia
69      Guatemala       112.37  2024  North America
70           Oman       109.99  2024           Asia
71       Bulgaria       108.42  2024         Europe
72      Venezuela       106.33  2024  South America

[73 rows x 4 columns]
Average GDP of top 5 countries by region
          Region  Top 5 Avg GDP (B USD)
0         Africa                238.182
1           Asia               6255.970
2         Europe               3318.112
3  North America               6946.500
4        Oceania                734.840
5  South America                79

/var/folders/rd/d9knhb6x6nj7rd817gqlcsr00000gn/T/ipykernel_5638/3158481998.py:109: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby('Region')


---

# Load에 있던 저장 프로세스를 Transform으로

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import json

# 로그 기록 함수
def log_message(message):
    timestamp = datetime.datetime.now().strftime('%Y-%b-%d-%H-%M-%S')
    with open('etl_project_log.txt', 'a') as log_file:
        log_file.write(f"{timestamp}, {message}\n")

# Extract
def extract_gdp_data(url):
    log_message("Starting Data Extraction")
    # HTTP 요청 후 파싱해서 데이터 추출
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'class': 'wikitable'})
    rows = table.find_all('tr')

    data = []
    for row in rows[1:]:  # 첫 번째 행은 헤더이므로 제외함
        cols = row.find_all('td')
        cleaned_cols = []
        for col in cols:
            # 'sup' 태그 제거 [n 1] 이런식으로 자꾸 떠서
            for sup in col.find_all('sup'):
                sup.decompose()
            cleaned_cols.append(col.text.strip())  # 텍스트 정리
        if cleaned_cols:  # 빈 행 제외
            try:
                country = cleaned_cols[0]  # 국가명
                gdp_raw = cleaned_cols[1]  # GDP 값 (Nominal GDP)
                gdp_year = cleaned_cols[2]
                gdp_cleaned = ''.join(filter(str.isdigit, gdp_raw.split('.')[0]))
                if gdp_cleaned:  # GDP 값이 유효한 경우만 추가
                    gdp = int(gdp_cleaned) / 1e3  # 단위를 1B USD로 변환
                    data.append({'Country': country, 'GDP (B USD)': round(gdp, 2), 'Year': gdp_year})
            except IndexError:
                # 예상치 못한 데이터 구조를 무시
                continue
    log_message("Data Extraction Completed")
    
    return data


# Transform
def transform_gdp_data(data, output_csv_file='gdp_by_country.csv', output_json_file='Countries_by_GDP.json'):
    log_message("Starting Data Transformation")
    # 데이터프레임 생성
    df = pd.DataFrame(data)

    # GDP 순으로 정렬
    df = df.sort_values(by='GDP (B USD)', ascending=False)
    
    # 국가별 Region 정보 매핑
    with open('country_region_table.json', 'r', encoding='utf-8') as region_file:
        region_data = json.load(region_file)
    df['Region'] = df['Country'].map(region_data)

    # 데이터 저장 (CSV 및 JSON 파일)
    try:
        df.to_csv(output_csv_file, index=False)
        log_message(f"CSV file saved as {output_csv_file}")
        
        data_as_dict = df.to_dict(orient='records')  # DataFrame을 딕셔너리 리스트로 변환
        with open(output_json_file, 'w', encoding='utf-8') as json_file:
            json.dump(data_as_dict, json_file, ensure_ascii=False, indent=4)
        log_message(f"JSON file saved as {output_json_file}")
    except Exception as e:
        log_message(f"Data Transformation Failed during saving: {str(e)}")
        raise

    log_message("Data Transformation Completed")
    return df

# Load
def load_gdp_data(df):
    log_message("Data Loading step is now minimal, performing any additional post-processing if required.")
    # 현재는 데이터 로드 관련 추가 작업이 없다면 빈 함수로 유지 가능
    pass

# GDP가 100B USD 이상인 국가만 필터링
def filtered_100USD(df):
    filtered_100 = df[df['GDP (B USD)'] >= 100]
    print("Countries with a GDP of over 100B USD")
    print(filtered_100)
    
    
 # Region별 상위 5개 국가의 GDP 평균 계산
def region_top5_calculate(df):
    region_top5_avg = (
        df.groupby('Region')
        .apply(lambda x: x.nlargest(5, 'GDP (B USD)')['GDP (B USD)'].mean())
        .reset_index(name='Top 5 Avg GDP (B USD)')
    )
    print("Average GDP of top 5 countries by region")
    print(region_top5_avg)

# 메인 ETL 함수
def etl_process():
    url = "https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)"
    try:
        log_message("ETL Process Started")
        # Extract
        data = extract_gdp_data(url)
        # Transform (includes saving)
        transformed_data = transform_gdp_data(data)
        # Load
        load_gdp_data(transformed_data)  
        log_message("ETL Process End Successfully")
        
        # GDP가 100B USD 이상인 국가만 필터링
        filtered_100USD(transformed_data)
        # Region별 상위 5개 국가의 GDP 평균 계산
        region_top5_calculate(transformed_data)
        
    except Exception as e:
        log_message(f"ETL Process Failed: {str(e)}")

if __name__ == "__main__":
    etl_process()

Countries with a GDP of over 100B USD
          Country  GDP (B USD)  Year         Region
0           World    115494.31  2025            NaN
1   United States     30337.16  2025  North America
2           China     19534.89  2025           Asia
3         Germany      4921.56  2025         Europe
4           Japan      4389.33  2025           Asia
..            ...          ...   ...            ...
68     Uzbekistan       112.65  2024           Asia
69      Guatemala       112.37  2024  North America
70           Oman       109.99  2024           Asia
71       Bulgaria       108.42  2024         Europe
72      Venezuela       106.33  2024  South America

[73 rows x 4 columns]
Average GDP of top 5 countries by region
          Region  Top 5 Avg GDP (B USD)
0         Africa                238.182
1           Asia               6255.970
2         Europe               3318.112
3  North America               6946.500
4        Oceania                734.840
5  South America                79

/var/folders/rd/d9knhb6x6nj7rd817gqlcsr00000gn/T/ipykernel_5638/547960222.py:94: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby('Region')


---

### 추가 요구 사항


#### 코드를 수정해서 아래 요구사항을 구현하세요.
- 추출한 데이터를 데이터베이스에 저장하세요. 'Countries_by_GDP'라는 테이블명으로 'World_Economies.db'라는 데이터 베이스에 저장되어야 합니다. 
    - 해당 테이블은 'Country', 'GDP_USD_billion'라는 어트리뷰트를 반드시 가져야 합니다.

데이터베이스는 sqlite3 라이브러리를 사용해서 만드세요.
- 필요한 모든 작업을 수행하는 'etl_project_gdp_with_sql.py' 코드를 작성하세요.

#### 화면 출력
- SQL Query를 사용해야 합니다.
    - GDP가 100B USD이상이 되는 국가만을 구해서 화면에 출력해야 합니다.
    - 각 Region별로 top5 국가의 GDP 평균을 구해서 화면에 출력해야 합니다

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import json
import sqlite3

# 로그 기록 함수
def log_message(message):
    timestamp = datetime.datetime.now().strftime('%Y-%b-%d-%H-%M-%S')
    with open('etl_project_log.txt', 'a') as log_file:
        log_file.write(f"{timestamp}, {message}\n")

# Extract
def extract_gdp_data(url):
    log_message("Starting Data Extraction")
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'class': 'wikitable'})
    rows = table.find_all('tr')

    data = []
    for row in rows[1:]:
        cols = row.find_all('td')
        cleaned_cols = []
        for col in cols:
            for sup in col.find_all('sup'):
                sup.decompose()
            cleaned_cols.append(col.text.strip())
        if cleaned_cols:
            try:
                country = cleaned_cols[0]
                gdp_raw = cleaned_cols[1]
                gdp_year = cleaned_cols[2]
                gdp_cleaned = ''.join(filter(str.isdigit, gdp_raw.split('.')[0]))
                if gdp_cleaned:
                    gdp = int(gdp_cleaned) / 1e3  # 단위를 1B USD로 변환
                    data.append({'Country': country, 'GDP (B USD)': round(gdp, 2), 'Year': gdp_year})
            except IndexError:
                continue
    log_message("Data Extraction Completed")
    return data

# Transform
def transform_gdp_data(data, output_csv_file='gdp_by_country.csv', output_json_file='Countries_by_GDP.json'):
    log_message("Starting Data Transformation")
    df = pd.DataFrame(data)
    df = df.sort_values(by='GDP (B USD)', ascending=False)
    with open('country_region_table.json', 'r', encoding='utf-8') as region_file:
        region_data = json.load(region_file)
    df['Region'] = df['Country'].map(region_data)

    # 데이터 저장 (CSV 및 JSON 파일)
    try:
        df.to_csv(output_csv_file, index=False)
        log_message(f"CSV file saved as {output_csv_file}")
        
        data_as_dict = df.to_dict(orient='records')
        with open(output_json_file, 'w', encoding='utf-8') as json_file:
            json.dump(data_as_dict, json_file, ensure_ascii=False, indent=4)
        log_message(f"JSON file saved as {output_json_file}")
    except Exception as e:
        log_message(f"Data Transformation Failed during saving: {str(e)}")
        raise

    log_message("Data Transformation Completed")
    return df

# Load to Database
def load_gdp_to_database(df, db_name='World_Economies.db', table_name='Countries_by_GDP'):
    log_message("Starting Data Loading to Database")
    try:
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()
        
        # 테이블 생성
        cursor.execute(f"""
            CREATE TABLE IF NOT EXISTS {table_name} (
                Country TEXT PRIMARY KEY,
                GDP_USD_billion REAL
            )
        """)
        
        # 데이터 삽입
        for _, row in df.iterrows():
            cursor.execute(f"""
                INSERT OR REPLACE INTO {table_name} (Country, GDP_USD_billion)
                VALUES (?, ?)
            """, (row['Country'], row['GDP (B USD)']))
        
        conn.commit()
        conn.close()
        log_message("Data Loading to Database Completed Successfully")
    except Exception as e:
        log_message(f"Data Loading to Database Failed: {str(e)}")
        raise

# SQL Query: GDP가 100B USD 이상인 국가 출력
def query_gdp_over_100(db_name='World_Economies.db', table_name='Countries_by_GDP'):
    log_message("Querying GDP over 100B USD")
    conn = sqlite3.connect(db_name)
    query = f"SELECT * FROM {table_name} WHERE GDP_USD_billion >= 100"
    result = pd.read_sql_query(query, conn)
    conn.close()
    print("Countries with a GDP of over 100B USD:")
    print(result)

# SQL Query: Region별 Top 5 국가의 GDP 평균
def query_region_top5_avg(df):
    log_message("Calculating Region-wise Top 5 Average GDP")
    region_top5_avg = (
        df.groupby('Region')
        .apply(lambda x: x.nlargest(5, 'GDP (B USD)')['GDP (B USD)'].mean())
        .reset_index(name='Top 5 Avg GDP (B USD)')
    )
    print("Average GDP of top 5 countries by region:")
    print(region_top5_avg)

# 메인 ETL 함수
def etl_process():
    url = "https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)"
    try:
        log_message("ETL Process Started")
        # Extract
        data = extract_gdp_data(url)
        # Transform
        transformed_data = transform_gdp_data(data)
        # Load to Database
        load_gdp_to_database(transformed_data)
        log_message("ETL Process Completed Successfully")

        # SQL Query Outputs
        query_gdp_over_100()
        query_region_top5_avg(transformed_data)
    except Exception as e:
        log_message(f"ETL Process Failed: {str(e)}")

if __name__ == "__main__":
    etl_process()

Countries with a GDP of over 100B USD:
          Country  GDP_USD_billion
0           World        115494.31
1   United States         30337.16
2           China         19534.89
3         Germany          4921.56
4           Japan          4389.33
..            ...              ...
68     Uzbekistan           112.65
69      Guatemala           112.37
70           Oman           109.99
71       Bulgaria           108.42
72      Venezuela           106.33

[73 rows x 2 columns]
Average GDP of top 5 countries by region:
          Region  Top 5 Avg GDP (B USD)
0         Africa                238.182
1           Asia               6255.970
2         Europe               3318.112
3  North America               6946.500
4        Oceania                734.840
5  South America                791.566


/var/folders/rd/d9knhb6x6nj7rd817gqlcsr00000gn/T/ipykernel_5638/3186562559.py:112: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby('Region')


---

In [1]:
import requests
import sqlite3
import datetime

def log_started():
    with open('etl_project_log.txt', 'a') as log_file:
        log_file.write("\n" + "="*50 + "\n")
        timestamp = datetime.datetime.now().strftime('%Y-%B-%d-%H-%M-%S')
        log_file.write(f"New execution at {timestamp}\n")
        log_file.write("\n" + "="*50 + "\n")


def log_message():
    pass
def save_gdp_data():
    pass
def extract_gdp_data():
    pass
def transform_gdp_data():
    pass
def load_gdp_data():
    pass



def etl_process():
    try:
        log_started()
        
    except:
        pass

if __name__ == "__main__":
    etl_process()

/Users/admin/Documents/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
